### Importing Libraries

In [21]:
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from datetime import timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

### Verifying the structure and dimentions of the data files

In [22]:
# Open the SSC dataset
ssc_ds = xr.open_dataset("Data/Processed_SSC_Data.nc")

# Print the SSC dataset
print("SSC Dataset:\n")
print(ssc_ds)
print("=" * 125)

# Open the wind dataset
wind_ds = xr.open_dataset("Data/Processed_Wind_Data.nc")

# Print the wind dataset
print("\nWind Dataset:\n")
print(wind_ds)
print("=" * 125)

# Close datasets after inspection
ssc_ds.close()
wind_ds.close()

SSC Dataset:

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 22, time: 2091)
Coordinates:
  * lat      (lat) float32 35.74 35.77 35.79 35.81 ... 36.21 36.23 36.26 36.28
  * lon      (lon) float32 13.92 13.96 14.0 14.04 ... 14.65 14.69 14.73 14.77
  * time     (time) datetime64[ns] 2021-01-01 2021-01-01T12:00:00 ... 2023-11-12
Data variables:
    u        (time, lat, lon) float64 ...
    v        (time, lat, lon) float64 ...
    stdu     (time, lat, lon) float64 ...
    stdv     (time, lat, lon) float64 ...
    cov      (time, lat, lon) float64 ...
    velo     (time, lat, lon) float64 ...
    head     (time, lat, lon) float64 ...
Attributes: (12/17)
    NC_GLOBAL.Title:                   Near-Real time Surface Ocean Velocity
    NC_GLOBAL.origin:                  BARK (measured);POZZ (measured);
    NC_GLOBAL.source:                  HF Radar Derived Surface Currents obta...
    NC_GLOBAL.history:                 08-Jun-2023 14:46:23
    NC_GLOBAL.grid_type:               REGULAR
    NC_

### Load Shapefile and Extract Coastline Boundaries

In [24]:
# Load the shapefile
shapefile_path = 'Data/land_sea_mask/MaltaCoastline.shp'
malta_coastline = gpd.read_file(shapefile_path)

# Assuming a roughly 3km per pixel resolution, calculate degree resolution approx
# For simplicity, we'll use the SSC dataset's extent and resolution as a reference
min_lon, max_lon = 13.9, 14.81
min_lat, max_lat = 35.6, 36.3

# Assuming the SSC dataset's grid dimensions to approximate grid size
grid_width = 22  # From SSC dataset lon dimension
grid_height = 25  # From SSC dataset lat dimension

# Determine grid resolution in degrees
lon_res = (max_lon - min_lon) / grid_width
lat_res = (max_lat - min_lat) / grid_height

# Create a transform for rasterization
transform = from_origin(min_lon, max_lat, lon_res, lat_res)

# Rasterize the shapefile
shapes = ((geom, 1) for geom in malta_coastline.geometry)
raster = rasterize(shapes=shapes, out_shape=(grid_height, grid_width), transform=transform, fill=0, all_touched=True, dtype='float32')

# Create longitude and latitude coordinates for the raster
lons = np.linspace(min_lon, max_lon - lon_res, grid_width)
lats = np.linspace(max_lat, min_lat + lat_res, grid_height)

# Convert the rasterized data to an xarray DataArray
land_sea_mask = xr.DataArray(raster, dims=['lat', 'lon'], coords={'lat': lats[::-1], 'lon': lons})

# Convert to xarray Dataset and assign variable name
ds = land_sea_mask.to_dataset(name='land_sea_mask')

# Define output path and save to NetCDF
output_nc_path = '/mnt/data/MaltaLandSeaMask.nc'
ds.to_netcdf(output_nc_path)

### Creating a FieldSet

In [25]:
# Initialize the FieldSet with ocean currents and wind data
fieldset = FieldSet.from_netcdf(
    filenames={
        "U": "Data/Processed_SSC_Data.nc",
        "V": "Data/Processed_SSC_Data.nc",
        "U_wind": "Data/Processed_Wind_Data.nc",
        "V_wind": "Data/Processed_Wind_Data.nc"
    },
    variables={
        "U": "u",
        "V": "v",
        "U_wind": "u10",
        "V_wind": "v10"
    },
    dimensions={
        "U": {"lon": "lon", "lat": "lat", "time": "time"},
        "V": {"lon": "lon", "lat": "lat", "time": "time"},
        "U_wind": {"lon": "longitude", "lat": "latitude", "time": "time"},
        "V_wind": {"lon": "longitude", "lat": "latitude", "time": "time"}
    },
    allow_time_extrapolation=True
)

# Load the land mask dataset and add it to the FieldSet
land_mask_ds = xr.open_dataset('Data/land_sea_mask/MaltaLandSeaMask.nc')
land_mask_field = Field.from_netcdf('Data/land_sea_mask/MaltaLandSeaMask.nc', variable='land_sea_mask', dimensions={'lon': 'lon', 'lat': 'lat'}, mesh='spherical', allow_time_extrapolation=True)
fieldset.add_field(land_mask_field)

MemoryError: Unable to allocate 4.05 GiB for an array with shape (30787, 35296) and data type float32

### Initializing the Particles

In [4]:
## Given polygon boundaries
#boundaries = {
#    'min_lon': 13.9, 'max_lon': 14.81,
#    'min_lat': 35.6, 'max_lat': 36.3
#}
#
## Calculate center of the polygon
#center_lon = (boundaries['min_lon'] + boundaries['max_lon']) / 2
#center_lat = (boundaries['min_lat'] + boundaries['max_lat']) / 2
#
## Define a range within which particles will be released around the center
## This will be a smaller area in the very center of your polygon
#range_lon = (boundaries['max_lon'] - boundaries['min_lon']) / 10  # 10% of the total longitude range
#range_lat = (boundaries['max_lat'] - boundaries['min_lat']) / 10  # 10% of the total latitude range
#
## Number of particles to release
#num_particles = 20
#
## Generate random positions within the central area
#lat_release = np.random.uniform(center_lat - range_lat, center_lat + range_lat, num_particles)
#lon_release = np.random.uniform(center_lon - range_lon, center_lon + range_lon, num_particles)
#
## Create a ParticleSet with JITParticle
#pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lon_release, lat=lat_release)

In [5]:
# Define polygon boundaries
boundaries = {
    'min_lon': 13.9, 'max_lon': 14.81,
    'min_lat': 35.6, 'max_lat': 36.3
}

# Adjusting the release points to be closer to the edges of the boundary
edge_buffer_lon = (boundaries['max_lon'] - boundaries['min_lon']) / 20  # 5% buffer from the longitude edges
edge_buffer_lat = (boundaries['max_lat'] - boundaries['min_lat']) / 20  # 5% buffer from the latitude edges

# Number of particles to release
num_particles = 20

# Generate positions close to the edges
lat_release_near_edges = np.concatenate([
    np.random.uniform(boundaries['min_lat'] + edge_buffer_lat, boundaries['min_lat'] + 2 * edge_buffer_lat, num_particles // 4),
    np.random.uniform(boundaries['max_lat'] - 2 * edge_buffer_lat, boundaries['max_lat'] - edge_buffer_lat, num_particles // 4)
])

lon_release_near_edges = np.concatenate([
    np.random.uniform(boundaries['min_lon'] + edge_buffer_lon, boundaries['min_lon'] + 2 * edge_buffer_lon, num_particles // 4),
    np.random.uniform(boundaries['max_lon'] - 2 * edge_buffer_lon, boundaries['max_lon'] - edge_buffer_lon, num_particles // 4)
])

# To ensure even distribution, shuffle the arrays
np.random.shuffle(lat_release_near_edges)
np.random.shuffle(lon_release_near_edges)

# Since we divided the particles into groups for each edge, we might need to adjust if num_particles is not a multiple of 4
# This is a simple fix to ensure we have exactly num_particles
lat_release = np.resize(lat_release_near_edges, num_particles)
lon_release = np.resize(lon_release_near_edges, num_particles)

# Create a ParticleSet with JITParticle positioned near the edges
pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lon_release, lat=lat_release)

### Define Simulation Parameters

In [6]:
# Define number of days for the simulation
runtime = timedelta(days=1046)  

#--------------------------------------------------------------------------------------------------------------------------

# Option 1: Fixed timestep (e.g., 10 minutes)
# dt = timedelta(minutes=12)

# Option 2: Timestep that aligns with your data intervals (e.g., 30 minutes or 1 hour)
#dt = timedelta(minutes=30)
dt = timedelta(hours=12)

# Option 3: If you want to use the exact timestep as your data
# dt = timedelta(hours=12)  # if your data interval is 12 hours

### Execute the Simulation

In [7]:
def CheckOutOfBounds(particle, fieldset, time):
    if particle.state == StatusCode.ErrorOutOfBounds:
        particle.delete()

def CheckError(particle, fieldset, time):
    if particle.state >= 50:  # This captures all Errors
        particle.delete()

# Execute the simulation
output_file = pset.ParticleFile(name="debris_simulation_output.zarr", outputdt=timedelta(minutes=720))  # Save output every hour
pset.execute(
    AdvectionRK4 + pset.Kernel(CheckOutOfBounds) + pset.Kernel(CheckError),  # Combine the advection kernel with the custom delete kernel
    runtime=runtime,
    dt=dt,
    output_file=output_file,
)

INFO: Output files are stored in debris_simulation_output.zarr.
100%|██████████| 90374400.0/90374400.0 [07:10<00:00, 209885.92it/s]


### Visualise

In [10]:
## Load the simulation output
#ds = xr.open_zarr("debris_simulation_output.zarr")
#
## Plot trajectories
#plt.figure(figsize=(10, 6))
#plt.scatter(ds.lon, ds.lat, s=1, c=ds.time, cmap='viridis', marker='o')
#plt.colorbar(label='Time')
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.title('Particle Trajectories')
#plt.show()

In [9]:
# Load the simulation output
ds = xr.open_zarr("debris_simulation_output.zarr")

# Define the geographic boundaries
extent = [13.9, 14.81, 35.6, 36.3]

# Create a GeoAxes with the PlateCarree projection
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(extent)

# Add land, coastlines, and the border of the map
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plot the particle trajectories
scatter = ax.scatter(ds.lon, ds.lat, s=1, c=ds.time, cmap='viridis', marker='o', transform=ccrs.PlateCarree())

# Add a colorbar and labels
cbar = plt.colorbar(scatter, ax=ax, label='Time')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Particle Trajectories')

plt.show()